In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive

/content/drive


In [3]:
cd MyDrive

/content/drive/MyDrive


In [4]:
cd driver-safety

/content/drive/MyDrive/driver-safety


In [5]:
cd Zero-DCE-1-master

/content/drive/MyDrive/driver-safety/Zero-DCE-1-master


In [6]:
cd Zero-DCE_code

/content/drive/MyDrive/driver-safety/Zero-DCE-1-master/Zero-DCE_code


In [7]:
import torch
import torch.nn as nn
import torchvision
import torch.backends.cudnn as cudnn
import torch.optim
import os
import sys
import argparse
import time
import dataloader
import model
import numpy as np
from torchvision import transforms
from PIL import Image
import glob
 
def lowlight(image_path):
    os.environ['CUDA_VISIBLE_DEVICES']='0'
    data_lowlight = Image.open(image_path)
 
 
 
    data_lowlight = (np.asarray(data_lowlight)/255.0)
 
 
    data_lowlight = torch.from_numpy(data_lowlight).float()
    data_lowlight = data_lowlight.permute(2,0,1)
    data_lowlight = data_lowlight.cuda().unsqueeze(0)
 
    DCE_net = model.enhance_net_nopool().cuda()
    DCE_net.load_state_dict(torch.load('snapshots/Epoch99.pth'))
    
    _,enhanced_image,_ = DCE_net(data_lowlight)
 
    result_path = 'a.jpg'
    torchvision.utils.save_image(enhanced_image, result_path)

In [8]:
import cv2
import dlib
import numpy as np
import os
from scipy.spatial import distance
from google.colab.patches import cv2_imshow
 
 
def calculate_eye(eye):
        A = distance.euclidean(eye[1], eye[5])
        B = distance.euclidean(eye[2], eye[4])
        C = distance.euclidean(eye[0], eye[3])
        eye_aspect_ratio = (A+B)/(2.0*C)
        return eye_aspect_ratio
 
def underlayer_calculate_lips(lips):
        A = distance.euclidean(lips[1], lips[7])
        B = distance.euclidean(lips[3], lips[5])
        C = distance.euclidean(lips[2], lips[6])
        D = distance.euclidean(lips[0], lips[4])
        lips_aspect_ratio = (A+B+C)/(3.0*D)
        return lips_aspect_ratio
 
def upperlayer_calculate_lips(lips):
        A = distance.euclidean(lips[1], lips[11])
        B = distance.euclidean(lips[2], lips[10])
        C = distance.euclidean(lips[3], lips[9])
        D = distance.euclidean(lips[4], lips[8])
        E = distance.euclidean(lips[5], lips[7])
        F = distance.euclidean(lips[0], lips[6])
        lips_aspect_ratio = (A+B+C+D+E)/(5.0*F)
        return lips_aspect_ratio
        
hog_face_detector = dlib.get_frontal_face_detector()
dlib_facelandmark = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
 
# Text settings
font = cv2.FONT_HERSHEY_COMPLEX
font_scale = 0.5
 
alpha=2
beta=50
 
# Initializations
sleep_score1=0
 
sleep_score2=0
frame_count=0
Initial_nose_tip_X=[]
count_X=0
average_nose_X = 0

In [35]:
cap = cv2.VideoCapture('Videos for Accuracy - lowlight/Abhishek/Blinking/abhishek_18')
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video  file")
 
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
#name = np.zeros((height,width,3), np.uint8) 
 
 
frames = []
 
 
# Read until video is completed
while(cap.isOpened()):
      
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
   
    # Display the resulting frame
 
    cv2.imwrite('x.jpg',frame)
    #saving the image to the same directory
 
    lowlight('x.jpg')
    #performing the enhancement on each frame
 
    frame = cv2.imread('a.jpg')
    #reading the enhanced frame from the folder
 
    frames.append(frame)
 
    os.remove('x.jpg')
    os.remove('a.jpg')
    #removing the frame and the enhanced frame from the folder each time
    
    height,width = frame.shape[:2]
    name = np.zeros((height,width,3), np.uint8)
    #creating a black filled rectangle which fills the entire frame
 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
    faces = hog_face_detector(gray)
    for face in faces:
        face_landmarks = dlib_facelandmark(gray,face)
  
        face_landmarks = dlib_facelandmark(gray,face)
 
        X=face_landmarks.part(34).x
        Y=face_landmarks.part(34).y
        
        
        if(len(Initial_nose_tip_X) < 10):
                Initial_nose_tip_X.append(X)
        if(len(Initial_nose_tip_X)==10):
                average_nose_X = int(sum(Initial_nose_tip_X)/10)
        leftEye = []
        rightEye = []
 
        if(X>(average_nose_X + 100)):
                count_X=count_X+1
        if(X<(average_nose_X - 75)):
                count_X=count_X+1
        if(X<(average_nose_X+100) and X>(average_nose_X - 75)):
                count_X=count_X-1
        if(count_X<0):
                count_X=0
        if(count_X > 40):
                cv2.putText(name, "ALERT!", (10,50), font, 2, (255, 255, 255), 4)
                
        if(count_X < 40):
                cv2.putText(name,'Pose Error:'+str(count_X),(width-250,height-160), font, 1,(255,255,255),1,cv2.LINE_AA)
 
        for n in range(36,42):
                x = face_landmarks.part(n).x
                y = face_landmarks.part(n).y
                leftEye.append((x,y))
                next_point = n+1
                if n == 41:
                        next_point = 36
                x2 = face_landmarks.part(next_point).x
                y2 = face_landmarks.part(next_point).y
 
        for n in range(42,48):
                x = face_landmarks.part(n).x
                y = face_landmarks.part(n).y
                rightEye.append((x,y))
                next_point = n+1
                if n == 47:
                        next_point = 42
                x2 = face_landmarks.part(next_point).x
                y2 = face_landmarks.part(next_point).y
 
 
        underlips = []
        for n in range(60,68):
                x = face_landmarks.part(n).x
                y = face_landmarks.part(n).y
                underlips.append((x,y))
                next_point = n+1
                if n == 67:
                        next_point = 60
                x2 = face_landmarks.part(next_point).x
                y2 = face_landmarks.part(next_point).y        
        upperlips = []
        for n in range(48,60):
                x = face_landmarks.part(n).x
                y = face_landmarks.part(n).y
                upperlips.append((x,y))
                next_point = n+1
                if n == 59:
                        next_point = 48
                x2 = face_landmarks.part(next_point).x
                y2 = face_landmarks.part(next_point).y
 
        left_eye = calculate_eye(leftEye)
        right_eye = calculate_eye(rightEye)
 
        EYE = (left_eye+right_eye)/2
        EYE = round(EYE,2)
 
        upper_lips = upperlayer_calculate_lips(upperlips)
 
        under_lips = underlayer_calculate_lips(underlips)
        
 
        LIPS = (under_lips + upper_lips) 
        LIPS = round(LIPS,2)
        
        if EYE < 0.24:
                sleep_score1=sleep_score1+1
                cv2.putText(name,"Closed eyes",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
        else:
                sleep_score1=sleep_score1-1
                cv2.putText(name,"Open eyes",(10,height-20), font, 1,(255, 255, 255),1,cv2.LINE_AA)
        
        if LIPS > .35:
                sleep_score2=sleep_score2+1
                cv2.putText(name,"Yawning",(10,height-80), font, 1,(255,255,255),1,cv2.LINE_AA)
                if (frame_count==0):
                        frame_count=frame_count+1   #it will initiate the counting of total number of frames once we will start yawning
                
        else:
                cv2.putText(name,"Not Yawning",(10,height-80), font, 1,(255,255,255),1,cv2.LINE_AA)
        
        if(sleep_score1<0):
                sleep_score1=0
 
        cv2.putText(name,'Blinking:'+str(sleep_score1),(width-200,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
        if(sleep_score1>5):
                #person is feeling sleepy so we beep the alarm
                cv2.putText(name, "ALERT!", (10,50), font, 2, (255, 255, 255), 4)
                
         
        cv2.putText(name,'Yawning:'+str(sleep_score2),(width-200,height-80), font, 1,(255,255,255),1,cv2.LINE_AA)
 
        if(sleep_score2)>50:         #For demonstration purpose it is 50 
                #person is feeling sleepy so we beep the alarm
                cv2.putText(name, "ALERT!", (10,50), font, 2, (255, 255, 255), 4)
 
              
        if(frame_count>0):
                frame_count=frame_count+1
 
        if (frame_count > 54000):          #54000=30*60*30
                sleep_score2=0
                frame_count=0
 
    
    cv2_imshow(name)
 
  
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
   
  # Break the loop
  else: 
    break
    
 
out = cv2.VideoWriter('abhishek_18_enhanced.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 10, (width, height))
for i in frames:
    out.write(i)
 
out.release()
 
# When everything done, release 
# the video capture object
cap.release()
   
# Closes all the frames
cv2.destroyAllWindows()

Error opening video  file
